In [ ]:
import numpy as np
import pandas as pd
#import matlplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import  RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_predict
from sklearn.utils import class_weight

from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D, AveragePooling2D, Conv1D, MaxPooling1D, AveragePooling1D
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.callbacks import TensorBoard, CSVLogger,  ModelCheckpoint
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler

# 1- DATA PREP

### Mode de lecture: liste des objets crées

Tous les jeux de données suivent ce code:
- '_std' = données standardisées avec StandardScaler
- '_MM' = données standardisées avec MinMaxScaler
- rien de spécifié alors aucune transfo

- **y** : catégories des 42 espèces encodées de 0 à 41

1- Utiliser la moyenne des réplicats:
- **X_train_m**, **X_test_m**
- **X_train_m_std**, **X_test_m_std**
- **X_train_m_MM**, **X_test_m_MM**
- **y_train** et **y_test**

2- Utiliser le troisème quantile des réplicats
- **X_train_q3**, **X_test_q3**
- **X_train_q3_std**, **X_test_q3_std**
- **X_train_q3_MM**, **X_test_q3_MM**
- **y_train_q3** et **y_test_q3**

3 - Utiliser tous les réplicats
- **X_test_all**, **X_train_all**
- **X_test_all_std**, **X_train_all_std**
- **X_test_all_MM**, **X_train_all_MM**
- **y_train_all**, **y_test_all**

4- Utiliser le Min et le Max des réplicats concatenate ensemble
- **X_train_mix**, **X_test_mix**
- **X_train_mix_std**, **X_test_mix_std**
- **X_train_mix_MM**, **X_test_mix_MM**
- **y_train_mix**, **y_test_mix**

5 - Prendre en compte le déséquilibre des classes:
- **weight** : prendre en compte le déséquilibre des classes dans la fonction de perte

In [ ]:
#chargement des données
data = np.genfromtxt('spectra-train.csv', delimiter = ";")
label_df = pd.read_csv('meta-train.csv', delimiter = ";")
key = label_df['species'].value_counts().keys().tolist()
valeur = label_df['species'].value_counts().tolist()
##encode y
y = label_df['species']
#encode y
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

#pour prendre en compte le déséquilibre des classes
#pour calculer les weights
weight = class_weight.compute_class_weight('balanced', np.unique(y), y)
weight = {i : weight[i] for i in range(42)}

# ---- MOYENNE des réplicats
#---------------------------------
# Création de la matrice avec la moyenne
data_mean=np.zeros((len(souches),data.shape[1]))
for x in range(len(souches)):
    data_mean[x,:]=np.mean(data[x*9:(x+1)*9, :], axis = 0)
#data_mean.shape#[346,627]

#creation d'une liste avec les indices des réplicats numéro 1:
z=0
index_R1= [None]* len(souches)
for i in range(len(souches)):
    #print(z)
    index_R1[i] = z
    z=z+9

# split data_mean
X_train_m, X_test_m, y_train, y_test = train_test_split(data_mean, y[index_R1], test_size = 0.15, random_state = 1, stratify = y[index_R1])

#standardization pour jeu avec les moyennes des réplicats
scaler = StandardScaler()
scaler.fit(X_train_m)
X_train_m_std = scaler.transform(X_train_m)
X_test_m_std = scaler.transform(X_test_m)

#standardization MinMax pour jeu avec les moyennes des réplicats
scalerMM = MinMaxScaler()
scalerMM.fit(X_train_m)
X_train_m_MM = scalerMM.transform(X_train_m)
X_test_m_MM = scalerMM.transform(X_test_m)


# ---- Q3 des réplicats
#---------------------------------
# Création de la matrice avec le troisième quantile
souches = pd.unique(label_df['strain'] )#nb de souches uniques
# Q3 pour chaque group de 9
data_Q3=np.zeros((len(souches),data.shape[1]))
for x in range(len(souches)):
    data_Q3[x,:]=np.quantile(data[x*9:(x+1)*9, :], 0.75, axis = 0)
#data_Q3.shape#[346,627]

# split data_q3
X_train_q3, X_test_q3, y_train_q3, y_test_q3 = train_test_split(data_Q3, y[index_R1], test_size = 0.15, random_state = 1, stratify = y[index_R1])

#standardization pour jeu avec les moyennes des réplicats
scaler = StandardScaler()
scaler.fit(X_train_q3)
X_train_q3_std = scaler.transform(X_train_q3)
X_test_q3_std = scaler.transform(X_test_q3)

#standardization MinMax pour jeu avec les moyennes des réplicats
scalerMM = MinMaxScaler()
scalerMM.fit(X_train_q3)
X_train_q3_MM = scalerMM.transform(X_train_q3)
X_test_q3_MM = scalerMM.transform(X_test_q3)

C:\Users\anais.rossetto\Anaconda3\lib\site-packages\sklearn\utils\validation.py:68: FutureWarning: Pass classes=[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41], y=[4 4 4 ... 9 9 9] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


### 3- Travailler avec tous les réplicats

In [ ]:
#je recharge tout depuis le debut pour être sûr chargement des données
data = np.genfromtxt('spectra-train.csv', delimiter = ";")
label_df = pd.read_csv('meta-train.csv', delimiter = ";")
key = label_df['species'].value_counts().keys().tolist()
valeur = label_df['species'].value_counts().tolist()
##encode y
y = label_df['species']
#encode y
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)
#ajout d'un index
label_df['index'] = np.arange(label_df.shape[0])
label_df.iloc[1:20,:]
#ajout d'un index idem sur data
z = np.arange(label_df.shape[0])
#res est data + une colonne ou on inscrit les index origniaux -> utile juste pour le split
res = np.concatenate((data,z[:,None]),axis=1)
#on crée une liste d'index tous les 9 pour avoir la première ligne de l'indice
#cette liste permet de stocker seulement une ligne parmis les 9 réplicats -> but éviter que le train test split sépare des réplicats
#initialisation de la liste
index9 = []

#index des replicat
for i in range(label_df.shape[0]):
    if i%9==0:
        index9.append(i)


X_train1, X_test1, y_train1, y_test1 = train_test_split(res[index9,:], y[index9], test_size = 0.15, random_state = 1, stratify = y[index9])

#il faut recuperer dans la dernière colonne de X tout les réplicats et les ajouter
index_train = (X_train1[:,627])
index_train = index_train.astype(int)#besoin transfo int pour utiliser en tant qu'indice
index_test = (X_test1[:,627])
index_test = index_test.astype(int)

#je fais ma première boucle en dehors comme ça évite initialisation et je peux concat dans ma boucle
X_train2 = data[index_train[0]:index_train[0]+9,:]
y_train2 = y[index_train[0]:index_train[0]+9]
X_test2 = data[index_test[0]:index_test[0]+9,:]
y_test2 = y[index_test[0]:index_test[0]+9]

#ensuite je les utilise pour reformer mes échantillons X_train et X_test depuis data seulement en utilisant les indices
for i in range(1,len(index_train)):
        X_train2 = np.concatenate((X_train2, data[index_train[i]:index_train[i]+9,:]), axis = 0)
        y_train2 = np.concatenate((y_train2, y[index_train[i]:index_train[i]+9]), axis = 0)

for i in range(1,len(index_test)):
        X_test2 = np.concatenate((X_test2, data[index_test[i]:index_test[i]+9,:]), axis = 0)
        y_test2 = np.concatenate((y_test2, y[index_test[i]:index_test[i]+9]), axis = 0)

X_train_all = X_train2
y_train_all = y_train2
X_test_all = X_test2
y_test_all = y_test2

# ------- !!!!!!!!  Commenter le code ci-dessous, si besoin d'avoir les y NON one HOT Encode
# ------------------------------------------------------------------------------------------------
#one hot encode y
encoder = OneHotEncoder()
y2 = y.reshape(-1, 1)
y_train_all = y_train_all.reshape(-1, 1)
y_test_all = y_test_all.reshape(-1, 1)
encoder.fit(y2)
y_train_all = encoder.transform(y_train_all).toarray()
y_test_all = encoder.transform(y_test_all).toarray()

#standardization pour jeu avec les moyennes des réplicats
scaler = StandardScaler()
scaler.fit(X_train_all)
X_train_all_std = scaler.transform(X_train_all)
X_test_all_std = scaler.transform(X_test_all)

#standardization MinMax pour jeu avec les moyennes des réplicats
scalerMM = MinMaxScaler()
scalerMM.fit(X_train_all)
X_train_all_MM = scalerMM.transform(X_train_all)
X_test_all_MM = scalerMM.transform(X_test_all)

### 4- Travailler avec le Min et le Max de chaque Réplicat

In [ ]:
#creation du vecteur concatenate le Min et le Max

# fonction qui boucle sur toutes les souches = de 0 à 9 et créer un sample
# calcul la somme de chaque ligne
# renvoie l'indice du min et du max
# concatene les deux lignes ensemble
# concatene chaque nouvelle ligne ensemble

# attention !!  cette fonction prend la ligne min et max sans transfo!

def MonMinMax(data):

    sample = np.zeros((9, data.shape[1]))
    for i in range(9):
        sample[i, :] = data[i, :]
    #on fait la sum par ligne
    somme = np.sum(sample, axis = 1)
    #recherhe du min et du max
    mini = np.argmin(somme)
    maxi = np.argmax(somme)

    matrice = np.concatenate((sample[mini, :], sample[maxi, :]), axis = 0)
    matrice = matrice.reshape(1, len(matrice))#ici on obtient la dim: (1, 1254)

    n=9
    #tant qu on atteind pas la dernière ligne on boucle
    while(n<data.shape[0]):
        sample = np.zeros((9, data.shape[1]))
        for i in range(9):
            sample[i, :] = data[i+n, :]
        #on fait la sum par ligne
        somme = np.sum(sample, axis = 1)
        #recherhe du min et du max
        mini = np.argmin(somme)
        maxi = np.argmax(somme)
        ligne = np.concatenate((sample[mini, :], sample[maxi, :]), axis = 0)
        ligne = ligne.reshape(1, len(ligne))
        matrice = np.concatenate((matrice, ligne), axis = 0)#(2, 1254)
        n = n+ 9
    return matrice

#transformation de X_train et X _test
X_train_mix = MonMinMax(X_train_all)
X_test_mix = MonMinMax(X_test_all)
temp = int(y_train_all.shape[0]/9) #permet d'avoir 0-9-18 ..
y_train_mix = y_train_all[index_R1[0]:index_R1[temp]]
temp = int(y_test_all.shape[0]/9)
y_test_mix = y_test_all[index_R1[0]:index_R1[temp]]

#standardization pour jeu avec les moyennes des réplicats
scaler = StandardScaler()
scaler.fit(X_train_mix)
X_train_mix_std = scaler.transform(X_train_mix)
X_test_mix_std = scaler.transform(X_test_mix)

#standardization MinMax pour jeu avec les moyennes des réplicats
scalerMM = MinMaxScaler()
scalerMM.fit(X_train_all)
X_train_mix_MM = scalerMM.transform(X_train_mix)
X_test_mix_MM = scalerMM.transform(X_test_mix)